[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/project-flow.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/laminlabs/lamin-usecases/main?labpath=lamin-usecases%2Fdocs%2Fproject-flow.ipynb)

# Project flow

LaminDB allows tracking data lineage on the entire project level.

Here, we walk through exemplified app uploads, pipelines & notebooks following [Schmidt _et al._, 2022](https://pubmed.ncbi.nlm.nih.gov/35113687/).

A CRISPR screen reading out a phenotypic endpoint on T cells is paired with scRNA-seq to generate insights into IFN-γ production.

These insights get linked back to the original data through the steps taken in the project to provide context for interpretation & future decision making.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/KQmzmmLOeBN0C8Ykitjn.svg" width="800">

:::{dropdown} More specifically: Why should I care about data flow?

Data flow tracks data sources & transformations to trace biological insights, verify experimental outcomes, meet regulatory standards, increase the robustness of research and optimize the feedback loop of team-wide learning iterations.

While tracking data flow is easier when it's governed by deterministic pipelines, it becomes hard when it's governed by interactive human-driven analyses.

LaminDB interfaces workflow mangers for the former and embraces the latter.

:::

```{toctree}
:maxdepth: 1
:hidden:

project-flow-scripts/hit-identification
project-flow-scripts/integrated-analysis
```

## Setup

Init a test instance:

Install the `lamindb` Python package:
```shell
pip install 'lamindb[jupyter,bionty,aws]'
```

In [ ]:
!lamin init --storage ./mydata

Import lamindb:

In [ ]:
import lamindb as ln
from IPython.display import Image, display

## Steps

In the following, we walk through exemplified steps covering different types of transforms ({class}`~docs:lamindb.Transform`).

```{note}

The full notebooks are in [this repository](https://github.com/laminlabs/rnd-demo).

```

### App upload of phenotypic data ![](https://img.shields.io/badge/app-444444)

Register data through app upload from wetlab by `testuser1`:

In [ ]:
# This function mimics the upload of artifacts via the UI
# In reality, you simply drag and drop files into the UI
def mock_upload_crispra_result_app():
    ln.setup.login("testuser1")
    transform = ln.Transform(name="Upload GWS CRISPRa result", type="upload")
    ln.track(transform=transform)
    output_path = ln.core.datasets.schmidt22_crispra_gws_IFNG(ln.settings.storage.root)
    output_file = ln.Artifact(
        output_path, description="Raw data of schmidt22 crispra GWS"
    )
    output_file.save()

mock_upload_crispra_result_app()

### Hit identification in notebook ![](https://img.shields.io/badge/notebook-444444)

Access, transform & register data in drylab by `testuser2` in notebook [hit-identification](project-flow-scripts/hit-identification).

In [ ]:
# the following mimics the integrated analysis notebook
# In reality, you would execute inside the notebook
import nbproject_test
from pathlib import Path

cwd = Path.cwd()
nbproject_test.execute_notebooks(cwd / "project-flow-scripts/hit-identification.ipynb", write=True)

Inspect data flow:

In [ ]:
artifact = ln.Artifact.filter(description="hits from schmidt22 crispra GWS").one()
artifact.view_lineage()

### Sequencer upload ![](https://img.shields.io/badge/script-444444)

Upload files from sequencer via script [chromium_10x_upload.py](https://github.com/laminlabs/lamin-usecases/blob/main/docs/project-flow-scripts/chromium_10x_upload.py):

In [ ]:
!python project-flow-scripts/chromium_10x_upload.py

### scRNA-seq bioinformatics pipeline ![](https://img.shields.io/badge/pipeline-444444)

Process uploaded files using a script or workflow manager: {doc}`docs:pipelines` and obtain 3 output files in a directory `filtered_feature_bc_matrix/`:

[cellranger.py](https://github.com/laminlabs/lamin-usecases/blob/main/docs/project-flow-scripts/cellranger.py)

In [ ]:
!python project-flow-scripts/cellranger.py

[postprocess_cellranger.py](https://github.com/laminlabs/lamin-usecases/blob/main/docs/project-flow-scripts/postprocess_cellranger.py)

In [ ]:
!python project-flow-scripts/postprocess_cellranger.py

Inspect data flow:

In [ ]:
output_file = ln.Artifact.filter(description="perturbseq counts").one()
output_file.view_lineage()

### Integrate scRNA-seq & phenotypic data ![](https://img.shields.io/badge/notebook-444444)

Integrate data in notebook [integrated-analysis](project-flow-scripts/integrated-analysis).

In [ ]:
# the following mimics the integrated analysis notebook
# In reality, you would execute inside the notebook
nbproject_test.execute_notebooks(cwd / "project-flow-scripts/integrated-analysis.ipynb", write=True)

## Review results

Let's load one of the plots:

In [ ]:
# track the current notebook as transform
ln.settings.transform.stem_uid = "1LCd8kco9lZU"
ln.settings.transform.version = "0"
ln.track()

In [ ]:
artifact = ln.Artifact.filter(key__contains="figures/matrixplot").one()
artifact.cache()

In [ ]:
display(Image(filename=artifact.path))

We see that the image artifact is tracked as an input of the current notebook. The input is highlighted, the notebook follows at the bottom:

In [ ]:
artifact.view_lineage()

Alternatively, we can also look at the sequence of transforms:

In [ ]:
transform = ln.Transform.search("Project flow").first()
transform.predecessors.df()

In [ ]:
transform.view_parents()

## Understand runs

We tracked pipeline and notebook runs through {class}`~docs:lamindb.core.run_context`, which stores a {class}`~docs:lamindb.Transform` and a {class}`~docs:lamindb.Run` record as a global context.

{class}`~lamindb.Artifact` objects are the inputs and outputs of runs. 

:::{dropdown} What if I don't want a global context?

Sometimes, we don't want to create a global run context but manually pass a run when creating an artifact:
```python
run = ln.Run(transform=transform)
ln.Artifact(filepath, run=run)
```

:::

:::{dropdown} When does an artifact appear as a run input?

When accessing an artifact via `cache()`, `load()` or `open()`, two things happen:

1. The current run gets added to `artifact.input_of`
2. The transform of that artifact gets added as a parent of the current transform

You can then switch off auto-tracking of run inputs if you set `ln.settings.track_run_inputs = False`: {doc}`docs:faq/track-run-inputs`

You can also track run inputs on a case by case basis via `is_run_input=True`, e.g., here:
```python
artifact.load(is_run_input=True)
```

:::

## Query by provenance

We can query or search for the notebook that created the artifact:

In [ ]:
transform = ln.Transform.search("GWS CRIPSRa analysis").first()

And then find all the artifacts created by that notebook:

In [ ]:
ln.Artifact.filter(transform=transform).df()

Which transform ingested a given artifact?

In [ ]:
artifact = ln.Artifact.filter().first()
artifact.transform

And which user?

In [ ]:
artifact.created_by

Which transforms were created by a given user?

In [ ]:
users = ln.User.lookup()

In [ ]:
ln.Transform.filter(created_by=users.testuser1).df()

Which notebooks were created by a given user?

In [ ]:
ln.Transform.filter(created_by=users.testuser1, type="notebook").df()

We can also view all recent additions to the entire database:

In [ ]:
ln.view()

In [ ]:
!lamin login testuser1
!lamin delete --force mydata
!rm -r ./mydata